# OPM Personnel Data and USAJobs API

#### Things to do going forward:

#### read data from opm website (CHECK)

#### generalize the cosine sim function (CHECK)

#### clean up the state mapping function

#### better process USAJobs data

### Reading Packages

In [17]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import date, datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_file,  output_notebook
from bokeh.sampledata.us_states import data as states
import io
pd.set_option('mode.chained_assignment', None)

### Get Directory

In [18]:
myDir=r"H:\_MyComputer\Documents\Git Repos\myOpmData\main\personnelData"
os.chdir(myDir)

### Functions

In [33]:
def makeDF(fileName, dictOfDF):
    dfName=fileName.split(".")[0]
    df=pd.read_csv(fileName, low_memory=False)
    dictOfDF[dfName]=df
    #print(dfName, df.columns)
    return(dictOfDF)

def subsetDF(df):
    subsetDF=df.loc[df['AGY']=="DD"]
    return(subsetDF)

def mergeSubset(mergeTable, allTable):
    mergedData=mergeTable.merge(allTable, left_on='AGYSUB', right_on='AGYSUB')
    return(mergedData)

def genericMerge(df1, df2):
    mergeColumns=[i for i in df1.columns if i in df2.columns]
    dfMerge=df1.merge(df2, left_on=mergeColumns, right_on=mergeColumns, how='left')
    return(dfMerge)

def tellMeAboutIt(df,agency):
    agencySubset=df.loc[df['AGYSUBT']==agency]
    topOccs=(agencySubset['OCCT'].value_counts(normalize=True).head(1))
    return(topOccs)

def occsByPercent(df,agency):
    agencySubset=df.loc[df['AGYSUBT']==agency]
    allOccs=(agencySubset['OCCT'].value_counts(normalize=True))
    return(allOccs)
    
def makeWordTemplate(df, AgencyName):
    document = Document()
    document.add_heading(f'Civilian Personnel Fact Sheet: {AgencyName}', level=1)
    document.add_heading(f"Average Salary: {'${:,.0f}'.format(df['SALARY'].mean())}", level=2)
    document.add_heading(f"Number of Employees: {'{:,.0f}'.format(len(df))}", level=2)
    #document.add_heading(f"Top Occupations: {'{:,.0f}'.format(len(df))}", level=2)
    data=getTopValuesPercents(df['OCCT'])
    #document=doctable(data, document)
    document.save('demo.docx')

def getTopValuesPercents(dfAndColumn, myNumber=5):
    seriesNumbers=dfAndColumn.value_counts().head(myNumber).map('{:,.0f}'.format)
    seriesPercents=dfAndColumn.value_counts(normalize=True).head(myNumber).map("{:.0%}".format)
    dfSum=pd.concat([seriesNumbers, seriesPercents], axis=1).reset_index()
    dfSum.columns=["Occupation", "Count", "Percent"]
    return(dfSum)

def doctable(data, document):
    table = document.add_table(rows=(data.shape[0]+1), cols=data.shape[1])
    for i, column in enumerate(data):
        for row in range(data.shape[0]-1) :
            table.cell(row, i).text = str(data[column][row])
    return(document)

def relevantJobs(wordsToFind, df):
    listOfJobs=[]
    for word in wordsToFind:
        listOfJobs.append([i for i in df['OCCT'].unique() if word in i.lower()])
    flat_list = [item for sublist in listOfJobs for item in sublist]
    allJobs=list(set(flat_list))
    return(allJobs)
        
def jobsByAgency(df, allJobs):
    df['selectedJobs']=0
    df.loc[df['OCCT'].isin(allJobs),'selectedJobs']=1
    allJobsByAgency=df[['AGYSUBT','selectedJobs']].groupby(['AGYSUBT']).agg(['mean', 'sum'])['selectedJobs'].sort_values(['sum'], ascending=False)
    return(allJobsByAgency.loc[allJobsByAgency['sum']>0])

def getRelJobsByAgency(wordsToFind, df):
    allJobs=relevantJobs(wordsToFind, df)
    allJobsByAgency=jobsByAgency(df, allJobs)
    return(allJobs, allJobsByAgency)

def getAllOccsbyAgency(df, cutoff):
    agencies=list(df['AGYSUBT'].unique())
    listOfAllOccs=[]
    for agency in agencies:
        results=occsByPercent(df,agency)
        listOfAllOccs.append([agency, results.index[0], results.values[0]])
    allOccs=pd.DataFrame(listOfAllOccs, columns=["Agency", "Occupation", "Percent"]).set_index("Agency")
    cutoffOccs=allOccs.loc[allOccs['Percent']>=cutoff]
    cutoffOccs['Percent']=cutoffOccs['Percent'].map("{:.0%}".format)
    return(cutoffOccs)

def occsByPercentOverall(df,occ):
    occSubset=df.loc[df['OCCT']==occ]
    allAgencies=(occSubset['AGYSUBT'].value_counts(normalize=True))
    return(allAgencies)

def getAllAgenciesByOcc(df, cutoff):
    occs=list(df['OCCT'].unique())
    listOfAllAgencies=[]
    for occ in occs:
        results=occsByPercentOverall(df,occ)
        listOfAllAgencies.append([occ, results.index[0], results.values[0]])
    allAgencies=pd.DataFrame(listOfAllAgencies, columns=["Occupation", "Agency", "Percent"]).set_index("Occupation")
    cutoffAgencies=allAgencies.loc[allAgencies['Percent']>=cutoff]
    cutoffAgencies['Percent']=cutoffAgencies['Percent'].map("{:.0%}".format)
    return(cutoffAgencies)

def overlapTwoSpecializedMeasures(results1, results2):
    overlap=results1.reset_index().merge(results2.reset_index(), right_on=['Occupation', 'Agency'], left_on=['Occupation', 'Agency'])
    overlap.columns=['Agency', 'Occupation', 'Percent at Agency', 'Percent Overall']
    return(overlap)

def putTogetherSpecializationOverlap(df, cutoffAtAgency, cutoffsOverAllAgencies):
    results1=getAllOccsbyAgency(ddDataCopy,cutoffAtAgency)
    results2=getAllAgenciesByOcc(ddDataCopy,cutoffsOverAllAgencies)
    overlap=overlapTwoSpecializedMeasures(results1, results2)
    return(overlap)

def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)

def keepNumbers(myString):
    justNumbers=''.join(i for i in myString if i.isdigit())
    return(justNumbers)

def connect():
    """ Forms the header parameters for authenticating with the v2 api. """
    headers = {'Authorization-Key': 'zm3i+Zu2AiT6+w3KqawsnwHaPkYexPVNaDJgYzcfWNc=',
 'Host': 'data.usajobs.gov',
 'User-Agent': 'abigail.haddad@gmail.com'}
    return headers
        
def format_dict(results, df_fixed):
    for j in range(0, len(results['SearchResult']['SearchResultItems'])):
        df_fixed=df_fixed.append(pd.DataFrame.from_dict(results['SearchResult']['SearchResultItems'][j]['MatchedObjectDescriptor'], orient='index').transpose())
        df_fixed=df_fixed.apply(pd.Series)
    return df_fixed

def current_search(agencyNum):
    """ Constructs v3 fuzzy searches with parameters outlined here:
    http://search.digitalgov.gov/developer/jobs.html """
    number=str(1)
    strAgencyNum=str(agencyNum)
    #base_url=f"https://data.usajobs.gov/api/search?Keyword={terms}&WhoMayApply=All&p="
    base_url=f"https://data.usajobs.gov/api/search?Organization=DD{strAgencyNum}&p="
    df_fixed=pd.DataFrame()
    results = requests.get(base_url+number, headers=connect()).json()
    #print(base_url+number)
    df_fixed=format_dict(results, df_fixed)
    if len(results['SearchResult']['SearchResultItems']) == 10:
            while results['SearchResult']['SearchResultCount']!= 0 and int(number)<101:
                number=str(int(number)+1)      
                results = requests.get(base_url+number, headers=connect()).json()
                #print(base_url+number)
                df_fixed=format_dict(results, df_fixed)
    return df_fixed

def pullAndUnzip(zipLine):
    r = requests.get(zipLink)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    
def cosinSimAcrossAgencies(df, variable):
    intermedDF=df[['AGYSUBT', variable, 'AGYTYP']].groupby(['AGYSUBT', variable]).count()
    intermedPct=intermedDF.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index()
    intermedPct.columns=['AGYSUBT', variable, 'PERCENT']
    reshapedDF=intermedPct.pivot(index='AGYSUBT', columns=variable, values='PERCENT')
    reshapedDF=reshapedDF.fillna(0) 
    listOfLists=[]
    for rowNumber1 in range(0, len(reshapedDF)):
        for rowNumber2 in range(0, len(reshapedDF)):
            if rowNumber1>rowNumber2:
                listToAppend=[reshapedDF.index[rowNumber1], reshapedDF.index[rowNumber2],cosine_similarity([reshapedDF.iloc[rowNumber1],reshapedDF.iloc[rowNumber2]])[0][1] ]
                listOfLists.append(listToAppend)
    dfCosine = pd.DataFrame(listOfLists)
    dfCosine.columns=['Org1', 'Org2', 'Similarity Index']
    dfCosine=dfCosine.sort_values('Similarity Index', ascending=False)
    return(dfCosine)


### Download OPM Files

In [20]:
pullAndUnzip("https://www.opm.gov/Data/Files/549/9e7f077c-a17a-46b6-a79a-d61c55d39841.zip")    

### Read In OPM Files

In [21]:
textFiles=[i for i in os.listdir() if i.lower().endswith(".txt")]
dictOfDF={}
for fileName in textFiles:
    dictOfDF=makeDF(fileName, dictOfDF)

### Subset Just Defense Department Data

In [22]:
ddAgencies=subsetDF(dictOfDF['DTagy'])
ddData= mergeSubset(ddAgencies, dictOfDF['FACTDATA_SEP2018'])

### Join With All Lookup Tables

In [23]:
ddDataCopy=ddData
textClean=[i.split(".")[0] for i in textFiles]
for fileName in textClean:
    ddDataCopy=genericMerge(ddDataCopy,dictOfDF[fileName])

### Look At Missing Data

In [24]:
for col in ddDataCopy.columns:
    count=ddDataCopy[col].count()
    if count<len(ddDataCopy):
        print(col, count/len(ddDataCopy))

EDLVL 0.9984271630561681
GSEGRD 0.6732715008431703
SALARY 0.9724591386690881


### Agency List

In [25]:
agencies=list(ddDataCopy['AGYSUBT'].unique())
for agency in agencies:
    print(agency)

DD01-OFFICE OF THE SECRETARY OF DEFENSE
DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF
DD04-DEFENSE INFORMATION SYSTEMS AGENCY
DD06-DEFENSE SECURITY COOPERATION AGENCY
DD07-DEFENSE LOGISTICS AGENCY
DD08-U.S. COURT OF APPEALS FOR THE ARMED FORCES
DD10-DEFENSE CONTRACT AUDIT AGENCY
DD12-DEFENSE SECURITY SERVICE
DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY
DD15-UNIFORMED SERVICES UNIVERSITY OF THE HEALTH SCIENCES
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY
DD21-WASHINGTON HEADQUARTERS SERVICES
DD23-OFFICE OF ECONOMIC ADJUSTMENT
DD25-DEFENSE LEGAL SERVICES AGENCY
DD26-OFFICE OF THE INSPECTOR GENERAL
DD27-MISSILE DEFENSE AGENCY
DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION
DD34-DEFENSE COMMISSARY AGENCY
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE
DD48-DEFENSE HUMAN RESOURCES ACTIVITY
DD53-DEFENSE POW/MIA ACCOUNTING AGENCY
DD60-DEFENSE HEALTH AGENCY
DD61-DEFENSE THREAT REDUCTION AGENCY
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY
DD65-PENTAGON FORCE PROTECTION AGENCY
DD68-DEPARTMENT OF DEFE

### Top Occupation by Agency

In [26]:
listsOfTopOccs=[]
for agency in agencies:
    results=tellMeAboutIt(ddDataCopy,agency)
    listsOfTopOccs.append([agency, results.index[0], results.values[0]])
    
listsOfTopOccs

topOccs=pd.DataFrame(listsOfTopOccs, columns=["Agency", "Top Occupation", "Percent"]).set_index("Agency")
topOccs['Percent']=topOccs['Percent'].map("{:.0%}".format)

In [27]:
topOccs.sort_values("Percent", ascending=False)

,Top Occupation,Percent
Agency,,
DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%
DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,80%
DD25-DEFENSE LEGAL SERVICES AGENCY,0905-GENERAL ATTORNEY,70%
DD23-OFFICE OF ECONOMIC ADJUSTMENT,1101-GENERAL BUSINESS AND INDUSTRY,70%
DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,69%
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,60%
DD71-DEFENSE MICROELECTRONICS ACTIVITY,0855-ELECTRONICS ENGINEERING,56%
DD08-U.S. COURT OF APPEALS FOR THE ARMED FORCES,0905-GENERAL ATTORNEY,53%
DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAGEMENT CENTER,0801-GENERAL ENGINEERING,50%


### Who Has Auditors?

In [28]:
auditOutput=getRelJobsByAgency(["audit"], ddDataCopy)
print(auditOutput[0])
auditOutput[1]

['0511-AUDITING']


,mean,sum
AGYSUBT,,
DD10-DEFENSE CONTRACT AUDIT AGENCY,0.899304,4394
DD26-OFFICE OF THE INSPECTOR GENERAL,0.369734,667
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.004647,60
DD07-DEFENSE LOGISTICS AGENCY,0.000739,20
DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.002500,15
DD27-MISSILE DEFENSE AGENCY,0.005924,15
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,0.000620,7
DD34-DEFENSE COMMISSARY AGENCY,0.000537,7
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,0.000110,2


### Who Has Accountants?

In [29]:
accountOutput=getRelJobsByAgency(["account"], ddDataCopy)
print(accountOutput[0])
accountOutput[1]

['0525-ACCOUNTING TECHNICIAN', '0510-ACCOUNTING']


,mean,sum
AGYSUBT,,
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.436837,5640
DD07-DEFENSE LOGISTICS AGENCY,0.006536,177
DD34-DEFENSE COMMISSARY AGENCY,0.006523,85
DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.006499,39
DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.011959,29
DD61-DEFENSE THREAT REDUCTION AGENCY,0.016937,23
DD27-MISSILE DEFENSE AGENCY,0.008294,21
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,0.001771,20
DD21-WASHINGTON HEADQUARTERS SERVICES,0.007877,18


### Who Has Human Resources?

In [30]:
humanOutput=getRelJobsByAgency(["human"], ddDataCopy)
print(humanOutput[0])
humanOutput[1]

['0299-HUMAN RESOURCES MANAGEMENT STUDENT TRAINEE', '0203-HUMAN RESOURCES ASSISTANCE', '0201-HUMAN RESOURCES MANAGEMENT']


,mean,sum
AGYSUBT,,
DD07-DEFENSE LOGISTICS AGENCY,0.022047,597
DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0.059063,319
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.019596,253
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,0.005877,107
DD21-WASHINGTON HEADQUARTERS SERVICES,0.035449,81
DD34-DEFENSE COMMISSARY AGENCY,0.004604,60
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,0.004428,50
DD27-MISSILE DEFENSE AGENCY,0.019352,49
DD26-OFFICE OF THE INSPECTOR GENERAL,0.023282,42


### Measures of Agency Specialization in an Occupation

#### Occupations Which Are Owned by An Agency

In [20]:
putTogetherSpecializationOverlap(ddDataCopy, 0, .5)

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD07-DEFENSE LOGISTICS AGENCY,6901-MISC WAREHOUSING AND STOCK HANDLING,15%,99%
1,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,85%
2,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0850-ELECTRICAL ENGINEERING,15%,65%
3,DD15-UNIFORMED SERVICES UNIVERSITY OF THE HEAL...,0401-GENERAL NATURAL RESOURCES MANAGEMENT AND ...,26%,92%
4,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,60%,98%
5,DD34-DEFENSE COMMISSARY AGENCY,2091-SALES STORE CLERICAL,23%,100%
6,DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0525-ACCOUNTING TECHNICIAN,25%,98%
7,DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,1910-QUALITY ASSURANCE,30%,77%
8,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,69%,74%
9,DD80-DEFENSE MEDIA ACTIVITY,1071-AUDIOVISUAL PRODUCTION,19%,70%


#### Occupations Which Dominate an Agency

In [21]:
putTogetherSpecializationOverlap(ddDataCopy, .5, 0)

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,85%
1,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,60%,98%
2,DD25-DEFENSE LEGAL SERVICES AGENCY,0905-GENERAL ATTORNEY,70%,22%
3,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,80%,40%
4,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,69%,74%


#### Overlap Between Those Groups

In [22]:
overlap=putTogetherSpecializationOverlap(ddDataCopy, .5, .5)
overlap

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,85%
1,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,60%,98%
2,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,69%,74%


### Are Other Agencies Doing 'Owned' Functions?

In [23]:
for occ in overlap["Occupation"].unique():
    print(occ)
    print(ddDataCopy.loc[ddDataCopy['OCCT']==occ][['AGYSUBT', 'OCCT']].groupby(['AGYSUBT']).count().sort_values("OCCT", ascending=False))
    print()

0511-AUDITING
                                                OCCT
AGYSUBT                                             
DD10-DEFENSE CONTRACT AUDIT AGENCY              4394
DD26-OFFICE OF THE INSPECTOR GENERAL             667
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE       60
DD07-DEFENSE LOGISTICS AGENCY                     20
DD04-DEFENSE INFORMATION SYSTEMS AGENCY           15
DD27-MISSILE DEFENSE AGENCY                       15
DD34-DEFENSE COMMISSARY AGENCY                     7
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY            7
DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF     2
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY      2
DD53-DEFENSE POW/MIA ACCOUNTING AGENCY             1
DD81-DEFENSE ACQUISITION UNIVERSITY                1

1701-GENERAL EDUCATION AND TRAINING
                                                     OCCT
AGYSUBT                                                  
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY       10918
DD69-NATIONAL DEFENSE UNIVERSITY 

### Measures of Similarity Across Agencies In Terms of Overall Occupational Profile

In [31]:
dfCosine=cosinSimAcrossAgencies(ddDataCopy, 'OCCT')
dfCosine.head(10)

,Org1,Org2,Similarity Index
0,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.972490
83,DD25-DEFENSE LEGAL SERVICES AGENCY,DD08-U.S. COURT OF APPEALS FOR THE ARMED FORCES,0.958434
171,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.946047
315,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD27-MISSILE DEFENSE AGENCY,0.938028
172,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.919315
209,DD53-DEFENSE POW/MIA ACCOUNTING AGENCY,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0.891202
380,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.878679
191,DD53-DEFENSE POW/MIA ACCOUNTING AGENCY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.861393
231,DD61-DEFENSE THREAT REDUCTION AGENCY,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.860704
190,DD53-DEFENSE POW/MIA ACCOUNTING AGENCY,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.859665


### Looking At Two Most Similar Agencies On This Metric

In [25]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF"]['OCCT'].value_counts().head()

0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM    352
0343-MANAGEMENT AND PROGRAM ANALYSIS             160
1515-OPERATIONS RESEARCH                          49
2210-INFORMATION TECHNOLOGY MANAGEMENT            42
0080-SECURITY ADMINISTRATION                      19
Name: OCCT, dtype: int64

In [26]:

ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD01-OFFICE OF THE SECRETARY OF DEFENSE"]['OCCT'].value_counts().head()

0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM    1173
0343-MANAGEMENT AND PROGRAM ANALYSIS              359
0130-FOREIGN AFFAIRS                              202
1515-OPERATIONS RESEARCH                          129
0560-BUDGET ANALYSIS                               80
Name: OCCT, dtype: int64

### We're Going to Measure Similarity Geographically

In [32]:
dfCosine=cosinSimAcrossAgencies(ddDataCopy, 'LOCT')
dfCosine.head(10)

,Org1,Org2,Similarity Index
316,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,1.000000
128,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999965
308,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999965
403,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,0.999954
394,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,0.999954
386,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999919
301,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999915
121,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999915
29,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999902
379,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999869


### Looking At Most Geographically Similar Agencies

In [28]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAGEMENT CENTER"]['LOCT'].value_counts().head()

51-VIRGINIA    24
Name: LOCT, dtype: int64

In [29]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION"]['LOCT'].value_counts().head()

51-VIRGINIA    122
Name: LOCT, dtype: int64

## This Is The End, Below Is In Progress: Mapping and USAJobs

In [30]:
def genStateDict():
    states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
    return(states)

In [31]:
stateDictionary=genStateDict()
stateUpper = {}
for k, v in stateDictionary.items():
    stateUpper[v.upper()] = k
stateUpper
stateSeries = pd.Series(stateUpper) 
stateDF=stateSeries.to_frame()
stateDF.columns=['State Abbreviation']

### State Name to Abbreviation

In [32]:
ddDataCopy['State Name']=ddDataCopy['LOCT'].str.split("-").str[1]

In [33]:
ddDataState=ddDataCopy.merge(stateDF, left_on='State Name', right_index=True, how='left')
#ddDataCopy['State Name'].value_counts()

### Good number of succesful merges

In [34]:
len(ddDataState['State Abbreviation'].value_counts())

53

### Construct State-Level Data

In [36]:
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]


stateDF=ddDataState[['State Abbreviation', 'LOCT']].groupby(['State Abbreviation']).count().reset_index()
stateDF.columns=['State', 'Count']
stateDF.head()
biggestNum=stateDF['Count'].max()
biggestNum
biggestNum/len(colors)

4604.666666666667

In [38]:
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")

state_xs = [states[code]["lons"] for code in states]
state_ys = [states[code]["lats"] for code in states]


state_colors = []
for state_id in states:
    try:
        rate = stateDF.loc[stateDF['State']==state_id]['Count']
        idx = int(rate/(biggestNum/len(colors))-1)
        state_colors.append(colors[idx])
    except KeyError:
        state_colors.append("black")

p = figure(title="DoD Employees by State", toolbar_location="left",
           plot_width=800, plot_height=500)

p.patches(state_xs, state_ys, fill_color=state_colors,
          line_color="#884444", line_width=2, line_alpha=0.3)

show(p)

### Let's Scrape Some USAJOBS Data


In [42]:
listOfPossibleAgencyNumbers=list(range(0, 101))

dfAllDod=pd.DataFrame()
for num in listOfPossibleAgencyNumbers:
    try:
        df_current=current_search(num)
        dfAllDod = pd.concat([dfAllDod, df_current])
        if len(df_current)>0:
            print(f' {num} is {len(df_current)}')
    except:
        pass


#range of DoD agencies is 0-100


 10 is 15
 11 is 28
 15 is 1
 16 is 77
 21 is 11
 25 is 1
 26 is 7
 27 is 3
 28 is 1
 34 is 73
 35 is 40
 48 is 1
 60 is 70
 61 is 8
 63 is 120
 65 is 4
 71 is 7
 80 is 12
 81 is 5


In [43]:
dfAllDod['DepartmentName'].value_counts()
dfAllDod['OrganizationName'].value_counts()

Defense Contract Management Agency                      120
Department of Defense Education Activity                 77
Defense Commissary Agency                                73
Defense Health Agency                                    70
Defense Finance and Accounting Service                   40
National Geospatial-Intelligence Agency                  28
Defense Contract Audit Agency                            15
Defense Media Activity                                   12
Washington Headquarters Services                         11
Defense Threat Reduction Agency                           8
Defense Microelectronics Activity                         7
Office of Inspector General                               7
Defense Acquisition University                            5
Pentagon Force Protection Agency                          4
Missile Defense Agency                                    3
Defense Human Resources Activity                          1
Defense Legal Services Agency           

In [44]:
for column in dfAllDod:
    print(column)
    print(dfAllDod[column].iloc[0])
    print("")


ApplicationCloseDate
2019-04-30

ApplyURI
['https://www.usajobs.gov:443/GetJob/ViewDetails/530648600?PostingChannelID=RESTAPI']

DepartmentName
Department of Defense

JobCategory
[{'Name': 'Auditing', 'Code': '0511'}]

JobGrade
[{'Code': 'GS'}]

OrganizationName
Defense Contract Audit Agency

PositionEndDate
2019-04-30

PositionFormattedDescription
[{'Label': 'Dynamic Teaser', 'LabelDescription': 'Hit highlighting for keyword searches.'}]

PositionID
DCAA-19-EA-10475764

PositionLocation
[{'LocationName': 'Wiesbaden, Germany', 'CountryCode': 'Germany', 'CityName': 'Wiesbaden, Germany', 'Longitude': 8.2423, 'Latitude': 50.0846}]

PositionLocationDisplay
Wiesbaden, Germany

PositionOfferingType
[{'Name': 'Permanent', 'Code': '15317'}]

PositionRemuneration
[{'MinimumRange': '90621.00', 'MaximumRange': '117810.00', 'RateIntervalCode': 'Per Year'}]

PositionSchedule
[{'Name': 'Full-Time', 'Code': '1'}]

PositionStartDate
2019-04-16

PositionTitle
SUPERVISORY AUDITOR

PositionURI
https://ww